In [2]:
import json
import pandas as pd
import numpy as np
import os

def files_to_df(dir_name):
    
    full_df = pd.DataFrame() # initialize full df compilation of files
    files = os.listdir(dir_name) # get list of files in directory
    print(f"{len(files)} files found in {dir_name}")

    for i in files:
        filename = os.path.join(dir_name, i)
        with open(filename, 'r') as f:
            data = json.loads(json.load(f))['data']
            df = pd.DataFrame(data)
        full_df = pd.concat([full_df, df], ignore_index = True) # join current json file to full df

    return full_df


def df_to_json(query):
    
    print(f"Converting json files in raw_data/{query} to dataframe.")
    df = files_to_df(f'raw_data/{query}')
    print(f"Shape of dataframe: {df.shape}")
        
    # convert dataframe back to json and export it
    export = df.to_json(f"full_data/{query}.json")
    print("Export complete.")
    
    return df

In [3]:
import time
import requests

def pause_report(length, random_delay, file_count, disp = True):
    
    pause_time = length*np.random.normal(1,random_delay)
    if disp: print(f"Downloaded {file_count} images. Pausing scraper for {round(pause_time,2)} seconds.")
    time.sleep(pause_time)

    
def download_files(query, df, file_start, file_end, short_pause = 2, long_pause = 60, random_delay = 0.25):

    file_count = 0 #number of files downloaded
    alrdy_exists = 0 #number of files that already exist and are skipped over
    skipped_urls = {} #urls that can't be connected to - need to retry at a later time
    
    if not os.path.exists(f'images/{query}'):
        print(f'Creating directory: images/{query}')
        os.makedirs(f'images/{query}')
    
    for i in range(file_start, file_end):

        if i >= len(df):
            print(f"Index {i} does not exist in dataframe.")
            print("End of dataframe has probably been reached. Process is terminating.")
            return 0
        
        filename = f"images/{query}/{df['hash_id'][i]}.jpg" # name the image using its hash_id

        if not os.path.exists(filename): # skip over files that have been downloaded
            
            try:
                img_url = df['smaller_square_cover_url'][i]
                print(f'Downloading {img_url} as {filename}')
                
                f = open(filename,'wb')
                f.write(requests.get(img_url).content)
                f.close()

                file_count += 1

                # take a short pause per url scraped
                pause_report(short_pause, random_delay, file_count)
                
            except:
                print("Cannot establish connection with the following row in df:")
                print(f"{i}:{img_url}")
                print("Skipping to next row")
                skipped_urls[i] = img_url #add key value pair to skipped_urls dict
                
                if len(skipped_urls) > 10:
                    print("More than 10 urls have had connection error.")
                    print(skipped_urls)
                    pause_report(300, random_delay, file_count) # pause for 5 minutes if more than 10 urls with connection error
                       
        else: 
            print(f"{filename} already exists. Skipping to next image url in df.")
            alrdy_exists += 1
            
        # take a long pause if 100 images have been downloaded
        if file_count > 0 and file_count%100 == 0:
            pause_report(long_pause, random_delay, file_count)
            
        # take a long pause x 2 if 500 images have been downloaded
        if file_count > 0 and file_count%500 == 0:
            pause_report(long_pause*2, random_delay, file_count)
      
    # Retry skipped urls
    print(f"Skipped the following urls because failed to make a connection:")
    print(skipped_urls)
    
    if len(skipped_urls) > 0:
        print("Retrying skipped urls.")
        
        for index, url in skipped_urls.items():
            try:
                filename = f"images/{query}/{df['hash_id'][index]}.jpg"
                print(f'Downloading {url} as {filename}.')

                f = open(filename,'wb')
                f.write(requests.get(img_url).content)
                f.close()

                file_count += 1

                # take a short pause per url scraped
                pause_report(short_pause, random_delay, file_count)

            except Exception as e:
                print("Still unable to connect to skipped urls. Please manually check for error.")
                print(e.message, e.args)
                return 0
    
    print(f"{file_count} images downloaded.")
    print(f"Skipped {alrdy_exists} images because already exists in database.")
    print("Process finished.")
    return 1
    

In [4]:
genres = ['cyberpunk','noir','horror','western','cartoon','steampunk']

df = df_to_json(genres[4])

Converting json files in raw_data/cartoon to dataframe.
133 files found in raw_data/cartoon
Shape of dataframe: (9973, 8)
Export complete.


In [5]:
download_files(genres[4], df, 8000, 10000)

Downloaded 1 images. Pausing scraper for 1.24 seconds.
Downloaded 2 images. Pausing scraper for 2.52 seconds.
Downloaded 3 images. Pausing scraper for 2.81 seconds.
Downloaded 4 images. Pausing scraper for 2.82 seconds.
Downloaded 5 images. Pausing scraper for 1.99 seconds.
Downloaded 6 images. Pausing scraper for 0.43 seconds.
Downloaded 7 images. Pausing scraper for 1.93 seconds.
Downloaded 8 images. Pausing scraper for 1.59 seconds.
Downloaded 9 images. Pausing scraper for 1.74 seconds.
Downloaded 10 images. Pausing scraper for 1.95 seconds.
Downloaded 11 images. Pausing scraper for 2.86 seconds.
Downloaded 12 images. Pausing scraper for 1.56 seconds.
Downloaded 13 images. Pausing scraper for 2.89 seconds.
Downloaded 14 images. Pausing scraper for 2.44 seconds.
Downloaded 15 images. Pausing scraper for 1.72 seconds.
Downloaded 16 images. Pausing scraper for 2.45 seconds.
Downloaded 17 images. Pausing scraper for 1.86 seconds.
Downloaded 18 images. Pausing scraper for 2.61 seconds.
D

Downloaded 38 images. Pausing scraper for 1.26 seconds.
Downloaded 39 images. Pausing scraper for 1.21 seconds.
Downloaded 40 images. Pausing scraper for 1.8 seconds.
Downloaded 41 images. Pausing scraper for 1.89 seconds.
Downloaded 42 images. Pausing scraper for 1.79 seconds.
Downloaded 43 images. Pausing scraper for 1.8 seconds.
Downloaded 44 images. Pausing scraper for 2.69 seconds.
Downloaded 45 images. Pausing scraper for 2.24 seconds.
Downloaded 46 images. Pausing scraper for 0.89 seconds.
Downloaded 47 images. Pausing scraper for 1.47 seconds.
Downloaded 48 images. Pausing scraper for 2.77 seconds.
Downloaded 49 images. Pausing scraper for 3.33 seconds.
Downloaded 50 images. Pausing scraper for 2.36 seconds.
Downloaded 51 images. Pausing scraper for 2.11 seconds.
Downloaded 52 images. Pausing scraper for 1.67 seconds.
Downloaded 53 images. Pausing scraper for 1.55 seconds.
Downloaded 54 images. Pausing scraper for 2.52 seconds.
Downloaded 55 images. Pausing scraper for 1.89 sec

Downloaded 75 images. Pausing scraper for 3.5 seconds.
Downloaded 76 images. Pausing scraper for 2.07 seconds.
Downloaded 77 images. Pausing scraper for 2.64 seconds.
Downloaded 78 images. Pausing scraper for 2.72 seconds.
Downloaded 79 images. Pausing scraper for 2.24 seconds.
Downloaded 80 images. Pausing scraper for 2.9 seconds.
Downloaded 81 images. Pausing scraper for 1.5 seconds.
Downloaded 82 images. Pausing scraper for 2.72 seconds.
Downloaded 83 images. Pausing scraper for 1.5 seconds.
Downloaded 84 images. Pausing scraper for 2.46 seconds.
Downloaded 85 images. Pausing scraper for 1.91 seconds.
Downloaded 86 images. Pausing scraper for 2.33 seconds.
Downloaded 87 images. Pausing scraper for 2.56 seconds.
Downloaded 88 images. Pausing scraper for 2.92 seconds.
Downloaded 89 images. Pausing scraper for 1.46 seconds.
Downloaded 90 images. Pausing scraper for 2.16 seconds.
Downloaded 91 images. Pausing scraper for 2.36 seconds.
Downloaded 92 images. Pausing scraper for 1.66 secon

Downloaded 111 images. Pausing scraper for 2.16 seconds.
Downloaded 112 images. Pausing scraper for 1.65 seconds.
Downloaded 113 images. Pausing scraper for 2.01 seconds.
Downloaded 114 images. Pausing scraper for 1.87 seconds.
Downloaded 115 images. Pausing scraper for 1.93 seconds.
Downloaded 116 images. Pausing scraper for 2.95 seconds.
Downloaded 117 images. Pausing scraper for 1.73 seconds.
Downloaded 118 images. Pausing scraper for 2.29 seconds.
Downloaded 119 images. Pausing scraper for 2.41 seconds.
Downloaded 120 images. Pausing scraper for 1.64 seconds.
Downloaded 121 images. Pausing scraper for 2.16 seconds.
Downloaded 122 images. Pausing scraper for 2.15 seconds.
Downloaded 123 images. Pausing scraper for 1.61 seconds.
Downloaded 124 images. Pausing scraper for 1.78 seconds.
Downloaded 125 images. Pausing scraper for 1.9 seconds.
Downloaded 126 images. Pausing scraper for 3.04 seconds.
Downloaded 127 images. Pausing scraper for 2.1 seconds.
Downloaded 128 images. Pausing sc

Downloaded 148 images. Pausing scraper for 1.77 seconds.
Downloaded 149 images. Pausing scraper for 2.46 seconds.
Downloaded 150 images. Pausing scraper for 2.43 seconds.
Downloaded 151 images. Pausing scraper for 2.59 seconds.
Downloaded 152 images. Pausing scraper for 1.99 seconds.
Downloaded 153 images. Pausing scraper for 1.44 seconds.
Downloaded 154 images. Pausing scraper for 2.23 seconds.
Downloaded 155 images. Pausing scraper for 2.48 seconds.
Downloaded 156 images. Pausing scraper for 2.32 seconds.
Downloaded 157 images. Pausing scraper for 2.63 seconds.
Downloaded 158 images. Pausing scraper for 1.99 seconds.
Downloaded 159 images. Pausing scraper for 2.55 seconds.
Downloaded 160 images. Pausing scraper for 1.79 seconds.
Downloaded 161 images. Pausing scraper for 2.06 seconds.
Downloaded 162 images. Pausing scraper for 2.79 seconds.
Downloaded 163 images. Pausing scraper for 2.51 seconds.
Downloaded 164 images. Pausing scraper for 1.58 seconds.
Downloaded 165 images. Pausing 

...

Downloaded 1844 images. Pausing scraper for 3.07 seconds.
Downloaded 1845 images. Pausing scraper for 1.57 seconds.
Downloaded 1846 images. Pausing scraper for 2.14 seconds.
Downloaded 1847 images. Pausing scraper for 1.07 seconds.
Downloaded 1848 images. Pausing scraper for 0.89 seconds.
Downloaded 1849 images. Pausing scraper for 1.15 seconds.
Downloaded 1850 images. Pausing scraper for 2.59 seconds.
Downloaded 1851 images. Pausing scraper for 1.72 seconds.
Downloaded 1852 images. Pausing scraper for 2.1 seconds.
Downloaded 1853 images. Pausing scraper for 1.72 seconds.
Downloaded 1854 images. Pausing scraper for 2.69 seconds.
Downloaded 1855 images. Pausing scraper for 2.24 seconds.
Downloaded 1856 images. Pausing scraper for 0.93 seconds.
Downloaded 1857 images. Pausing scraper for 1.78 seconds.
Downloaded 1858 images. Pausing scraper for 1.67 seconds.
Downloaded 1859 images. Pausing scraper for 1.79 seconds.
Downloaded 1860 images. Pausing scraper for 1.49 seconds.
Downloaded 1861

Downloaded 1881 images. Pausing scraper for 1.65 seconds.
Downloaded 1882 images. Pausing scraper for 2.52 seconds.
Downloaded 1883 images. Pausing scraper for 2.16 seconds.
Downloaded 1884 images. Pausing scraper for 2.23 seconds.
Downloaded 1885 images. Pausing scraper for 2.35 seconds.
Downloaded 1886 images. Pausing scraper for 2.17 seconds.
Downloaded 1887 images. Pausing scraper for 2.62 seconds.
Downloaded 1888 images. Pausing scraper for 2.17 seconds.
Downloaded 1889 images. Pausing scraper for 2.8 seconds.
Downloaded 1890 images. Pausing scraper for 2.9 seconds.
Downloaded 1891 images. Pausing scraper for 2.55 seconds.
Downloaded 1892 images. Pausing scraper for 2.44 seconds.
Downloaded 1893 images. Pausing scraper for 2.36 seconds.
Downloaded 1894 images. Pausing scraper for 2.23 seconds.
Downloaded 1895 images. Pausing scraper for 2.98 seconds.
Downloaded 1896 images. Pausing scraper for 1.4 seconds.
Downloaded 1897 images. Pausing scraper for 2.74 seconds.
Downloaded 1898 i

Downloaded 1917 images. Pausing scraper for 1.91 seconds.
Downloaded 1918 images. Pausing scraper for 1.4 seconds.
Downloaded 1919 images. Pausing scraper for 2.03 seconds.
Downloaded 1920 images. Pausing scraper for 1.85 seconds.
Downloaded 1921 images. Pausing scraper for 2.35 seconds.
Downloaded 1922 images. Pausing scraper for 1.94 seconds.
Downloaded 1923 images. Pausing scraper for 1.9 seconds.
Downloaded 1924 images. Pausing scraper for 2.6 seconds.
Downloaded 1925 images. Pausing scraper for 1.81 seconds.
Downloaded 1926 images. Pausing scraper for 1.84 seconds.
Downloaded 1927 images. Pausing scraper for 1.98 seconds.
Downloaded 1928 images. Pausing scraper for 1.22 seconds.
Downloaded 1929 images. Pausing scraper for 2.29 seconds.
Downloaded 1930 images. Pausing scraper for 1.71 seconds.
Downloaded 1931 images. Pausing scraper for 1.57 seconds.
Downloaded 1932 images. Pausing scraper for 2.64 seconds.
Downloaded 1933 images. Pausing scraper for 1.06 seconds.
Downloaded 1934 i

Downloaded 1953 images. Pausing scraper for 1.05 seconds.
Downloaded 1954 images. Pausing scraper for 2.23 seconds.
Downloaded 1955 images. Pausing scraper for 1.91 seconds.
Downloaded 1956 images. Pausing scraper for 1.6 seconds.
Downloaded 1957 images. Pausing scraper for 2.47 seconds.
Downloaded 1958 images. Pausing scraper for 1.76 seconds.
Downloaded 1959 images. Pausing scraper for 2.19 seconds.
Downloaded 1960 images. Pausing scraper for 2.61 seconds.
Downloaded 1961 images. Pausing scraper for 1.27 seconds.
Downloaded 1962 images. Pausing scraper for 1.77 seconds.
Downloaded 1963 images. Pausing scraper for 1.93 seconds.
Index 9973 does not exist in dataframe.
End of dataframe has probably been reached. Process is terminating.


0

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

plt.figure(figsize=(20,6))
for i in range(5):
    plt.subplot(1,5,i+1)
    img = mpimg.imread(f"images/{query}/{df['hash_id'][i+60]}.jpg")
    plt.title(f"{df['title'][i]}")
    plt.imshow(img)
    
    